<a href="https://colab.research.google.com/github/Harmswing18/Volleyballplayer_Tracking/blob/main/Kopie_von_Playertracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Github/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Github


In [4]:
!pip -q install ultralytics supervision opencv-python tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.2/207.2 kB 9.3 MB/s eta 0:00:00


In [2]:
import torch
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)

Using device: cpu


In [9]:
!ffmpeg -y -err_detect ignore_err -i "/content/drive/MyDrive/Github/Kopie von test_converted.mp4" \
  -c copy -movflags faststart "/content/local_input_fixed.mp4"


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [ ]:
import os, cv2, numpy as np, torch
from ultralytics import YOLO
import supervision as sv

# =====================
# Konfiguration
# =====================
NUM_MODEL_PATH = "/content/drive/MyDrive/Github/weights/Kopie von best.pt"     # dein Rückennummernmodell
VIDEO_PATH_DRIVE = "/content/drive/MyDrive/Github/Kopie von Testsnippet Karlsruhe noch kleiner – Mit Clipchamp erstellt.mp4"       # Inputvideo in Drive
VIDEO_PATH = "/content/local_input.mp4"                               # lokale Kopie
OUT_PATH   = "/content/debug_tracked.mp4"                             # Ausgabevideo

!cp "$VIDEO_PATH_DRIVE" "$VIDEO_PATH"

DEVICE    = "cuda:0" if torch.cuda.is_available() else "cpu"
NUM_IMGSZ = 1280
PER_IMGSZ = 960
NUM_CONF  = 0.1
PER_CONF  = 0.25

TARGET_CLASSES = ["Libero-Nr.-15", "Opposite-Nr.-4"]
VOTE_WINDOW = 5   # wie viele Frames für Mehrheitsvoting

print("Using device:", DEVICE)

# =====================
# Hilfsfunktionen
# =====================
def keep_one_per_class(det_list):
    """Behält pro Klasse die Detection mit höchster Confidence."""
    best = {}
    for bbox, cls_name, conf in det_list:
        if cls_name not in best or conf > best[cls_name][2]:
            best[cls_name] = (bbox, cls_name, conf)
    return list(best.values())

def compute_iou(boxA, boxB):
    """IOU zwischen zwei Boxen [x1,y1,x2,y2]."""
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    inter = max(0, xB - xA) * max(0, yB - yA)
    if inter == 0: return 0.0
    areaA = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    areaB = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    return inter / float(areaA + areaB - inter)

# =====================
# Modelle laden
# =====================
num_model = YOLO(NUM_MODEL_PATH)
per_model = YOLO("yolov8n.pt")
num_names = num_model.names
per_names = per_model.names

# =====================
# Video Setup
# =====================
cap = cv2.VideoCapture(VIDEO_PATH)
assert cap.isOpened(), f"Kann Video nicht öffnen: {VIDEO_PATH}"

fps = cap.get(cv2.CAP_PROP_FPS) or 25
W   = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
H   = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print("Video Info:", W, "x", H, "FPS:", fps)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(OUT_PATH, fourcc, fps, (W, H))
assert out.isOpened(), "Writer nicht geöffnet"

# =====================
# Tracker + Annotatoren
# =====================
player_tracker  = sv.ByteTrack()
box_annotator   = sv.BoxAnnotator(thickness=2)
label_annotator = sv.LabelAnnotator(text_thickness=2, text_scale=0.7)

tid_to_number = {}       # TrackID -> stabilisierte Nummer
tid_to_votes  = {}       # TrackID -> Liste letzter Nummern für Voting
tid_to_conf   = {}       # TrackID -> Confidence bei Erstzuordnung

frame_idx = 0

while True:
    ok, frame = cap.read()
    if not ok or frame is None:
        print("Ende erreicht oder kein Frame.")
        break

    frame_out = frame.copy()

    # -----------------
    # Nummern-Detektion
    # -----------------
    r_num = num_model(frame, imgsz=NUM_IMGSZ, conf=NUM_CONF, device=DEVICE, verbose=False)[0]
    raw_num = sv.Detections.from_ultralytics(r_num)

    filtered = []
    for i in range(len(raw_num.xyxy)):
        cls_id = int(raw_num.class_id[i])
        cls_name = num_names[cls_id]
        if cls_name in TARGET_CLASSES:
            bbox = raw_num.xyxy[i]
            conf = float(raw_num.confidence[i])
            filtered.append((bbox, cls_name, conf))

    filtered = keep_one_per_class(filtered)

    # -----------------
    # Personen-Detektion
    # -----------------
    r_per = per_model(frame, imgsz=PER_IMGSZ, conf=PER_CONF, device=DEVICE, verbose=False)[0]
    per_dets_all = sv.Detections.from_ultralytics(r_per)
    mask = [per_names[int(cid)] == "person" for cid in per_dets_all.class_id]
    per_dets = per_dets_all[np.array(mask)]

    # -----------------
    # Tracking
    # -----------------
    tracks = player_tracker.update_with_detections(per_dets) if len(per_dets) > 0 else sv.Detections.empty()

    # -----------------
    # Nummern -> Tracks (IOU Matching + Voting)
    # -----------------
    if len(tracks) > 0 and len(filtered) > 0:
        for (bbox, cls_name, conf) in filtered:
            best_tid, best_iou = None, 0
            for tbox, tid in zip(tracks.xyxy, tracks.tracker_id):
                iou = compute_iou(bbox, tbox)
                if iou > best_iou:
                    best_iou = iou
                    best_tid = int(tid)
            if best_tid is not None and best_iou > 0.1:  # min. IOU Schwelle
                # Voting eintragen
                tid_to_votes.setdefault(best_tid, []).append(cls_name)
                if len(tid_to_votes[best_tid]) > VOTE_WINDOW:
                    tid_to_votes[best_tid].pop(0)
                # Stabilisierte Nummer = Mehrheitsentscheidung
                voted = max(set(tid_to_votes[best_tid]), key=tid_to_votes[best_tid].count)
                tid_to_number[best_tid] = voted
                tid_to_conf[best_tid]   = max(conf, tid_to_conf.get(best_tid, 0))

    # -----------------
    # Primary Track pro Klasse (höchste Confidence)
    # -----------------
    number_to_primary_tid = {}
    for t_xyxy, t_id in zip(tracks.xyxy, tracks.tracker_id):
        t_id = int(t_id)
        num_txt = tid_to_number.get(t_id, None)
        if num_txt in TARGET_CLASSES:
            conf = tid_to_conf.get(t_id, 0)
            if (num_txt not in number_to_primary_tid) or (conf > number_to_primary_tid[num_txt][1]):
                number_to_primary_tid[num_txt] = (t_id, conf)

    # -----------------
    # Rendering nur Primary Tracks
    # -----------------
    draw_labels, draw_tracks = [], []
    for t_xyxy, t_id in zip(tracks.xyxy, tracks.tracker_id):
        t_id = int(t_id)
        num_txt = tid_to_number.get(t_id, None)
        if not num_txt:
            continue
        primary_tid = number_to_primary_tid.get(num_txt, (t_id,0))[0]
        if t_id != primary_tid:
            continue
        draw_labels.append(f"#{num_txt}")
        draw_tracks.append(t_xyxy)

    if draw_tracks:
        draw_dets = sv.Detections(
            xyxy=np.array(draw_tracks, dtype=float),
            confidence=None,
            class_id=np.zeros((len(draw_tracks),), dtype=int)
        )
        frame_out = box_annotator.annotate(frame_out, detections=draw_dets)
        frame_out = label_annotator.annotate(frame_out, detections=draw_dets, labels=draw_labels)

    cv2.putText(frame_out,
                f"frame={frame_idx} num={len(filtered)} per={len(per_dets)} tracks={len(tracks)}",
                (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,255,255), 2)

    out.write(frame_out)
    frame_idx += 1

cap.release()
out.release()
print("Fertig:", OUT_PATH, "size:", os.path.getsize(OUT_PATH))


Using device: cpu
Video Info: 1920 x 1080 FPS: 30.0


In [ ]:
from google.colab import files
files.download(OUT_PATH)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
files.download(OUT_PATH)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>